# Chapter 7 - Ensemble Learning

# Exercise 1

* This notebook is created to execute and experiment with the exercise from Chapter 7 of "Hands-on ML" book. 
* The exercise details are as follows, 
    * Load the MNIST dataset (introduced in Chapter 3), and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing).
    * Then train various classifiers, such as a random forest classifier, an extra-trees classifier, and an SVM classifier. 
    * Next, try to combine them into an ensemble that outperforms each individual classifier on the validation set, using soft or hard voting. 
    * Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

## Plan
* After experimenting in previous we realized that the model with Augmented dataset performs a lot better than the default regular dataset. So thats what I want to use for this exercise. 
* The plan is as follows,
    * Split the data into training set, validation set and test set. 
    * Augment the entire dataset (instead of just the training set as before) and save it as CSV for future use.     
    * Train the following classifiers, (we've trained them in previous notebooks)
        * Logistic Regression
        * SVC
        * Random Foreset
        * KNN
        * Extra Trees Classifier
        * Gradient Boosting Classifier
    * We'll just try `Hard Voting` for now since for `Soft Voting` we need probabilities and calculating those for `SVM` can be time consuming. 
    * For the first version I think I'll train them using default params and then may be try training them using best params I got in previous notebooks. 
    * Compare the performance against individual classifiers and best classifier in previous notebook



## Import Libraries

In [1]:
from sklearn.preprocessing import Binarizer, OneHotEncoder, MinMaxScaler, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict,GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from pathlib import Path
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import ConfusionMatrixDisplay, f1_score, roc_auc_score, roc_curve, accuracy_score
from sklearn.dummy import DummyClassifier


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import joblib
import json
import gdown
import os
import sys
import tqdm
import time
import warnings
from huggingface_hub import login, HfApi,list_repo_files, hf_hub_download
import os
import sys
import dotenv
from pathlib import Path
import tqdm

## Setup Hugging Face

In [2]:
dotenv.load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

## login huggingface user
if HF_TOKEN is None:
    print("Please set the HF_TOKEN environment variable. This is you hugging face token")
else:
    print("Logging in...")
    login(HF_TOKEN)
    
api = HfApi()
user = api.whoami()
user_name = user['name']
print(f"Logged in as {user_name}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logging in...
Logged in as gaurangdave


## Download Models & Data

In [3]:
# # Get the directory of the notebook
## this is root directory of jupyterlab
notebook_dir = os.getcwd()

# from api.utils.common import augment_dataset,download_data_from_gdrive
project_root = os.path.abspath(os.path.join(notebook_dir , "mnist_digits_recognition"))  
model_root = os.path.join(project_root, "models")
data_root = os.path.join(project_root, "data")

## hugging face repo id
model_name = "mnist_digits_recognition"
repo_id = f"{user_name}/{model_name}"


In [4]:
all_models = api.list_repo_files(repo_id, repo_type="model")

print(f"Models in {repo_id}:")
for model in all_models:
    ## download to tmp folder
    file_path = hf_hub_download(repo_id=repo_id, filename=model, token=HF_TOKEN, local_dir=model_root, repo_type="model")
    ## move to model folder
    os.rename(file_path, os.path.join(model_root, model))
    print(f"Downloaded {model} to {file_path}")

print(f"✅ All files downloaded to {model_root}")

Models in gaurangdave/mnist_digits_recognition:
Downloaded .gitattributes to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/.gitattributes
Downloaded default_logistic_regression_probabilites.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/default_logistic_regression_probabilites.csv
Downloaded ensemble/extra_trees_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/extra_trees_model.pkl
Downloaded ensemble/gradient_boosting_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/gradient_boosting_model.pkl
Downloaded ensemble/knn_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/knn_model.pkl


logistic_regression_model.pkl:   0%|          | 0.00/109k [00:00<?, ?B/s]

Downloaded ensemble/logistic_regression_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/logistic_regression_model.pkl
Downloaded ensemble/random_forest_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/random_forest_model.pkl
Downloaded ensemble/soft_voting_classifier_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/soft_voting_classifier_model.pkl
Downloaded ensemble/svc_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/svc_model.pkl
Downloaded ensemble/tuned_knn_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/tuned_knn_model.pkl
Downloaded ensemble/tuned_logistic_regression_model.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/tuned_logistic_regression_model.pkl
Downloaded ensemble/tuned_logistic_regression_v0.pkl to /home/gaurangdave/jupyterlab/mnist_digits_recognition/models/ensemble/tu

In [5]:
all_data = api.list_repo_files(repo_id, repo_type="dataset")
    
print(f"Datasets in {repo_id}:")
for dataset in all_data:
    ## download to tmp folder
    file_path = hf_hub_download(repo_id=repo_id, filename=dataset, token=HF_TOKEN, local_dir=data_root, repo_type="dataset")
    ## move to model folder
    os.rename(file_path, os.path.join(data_root, dataset))
    print(f"Downloaded {dataset} to {file_path}")

print(f"✅ All files downloaded to {model_root}")

    

Datasets in gaurangdave/mnist_digits_recognition:
Downloaded .gitattributes to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/.gitattributes
Downloaded augmented_ensemle_learning_mnist_data.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/augmented_ensemle_learning_mnist_data.csv
Downloaded augmented_mnist_data.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/augmented_mnist_data.csv
Downloaded augmented_train_X.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/augmented_train_X.csv
Downloaded augmented_train_Y.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/augmented_train_Y.csv
Downloaded mnist_test_set.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/mnist_test_set.csv
Downloaded mnist_train_set.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/mnist_train_set.csv
Downloaded raw_mnist_data.csv to /home/gaurangdave/jupyterlab/mnist_digits_recognition/data/raw_mnist_data.cs

## Import Custom Scripts

In [4]:
def shift_image(image, direction):
    """
    Shift an MNIST image in a given direction.

    Args:
        image (np.ndarray): 1D array of 784 pixels (28x28 image).
        direction (str): One of 'up', 'down', 'left', 'right'.

    Returns:
        np.ndarray: Shifted 1D image (flattened).
    """
    # Reshape the flat image to 28x28
    image_2d = image.reshape(28, 28)
    # Create an empty 28x28 array filled with zeros
    shifted = np.zeros_like(image_2d)

    # Perform the shift based on the direction
    if direction == "up":
        shifted[:-1, :] = image_2d[1:, :]  # Shift rows up
    elif direction == "down":
        shifted[1:, :] = image_2d[:-1, :]  # Shift rows down
    elif direction == "left":
        shifted[:, :-1] = image_2d[:, 1:]  # Shift columns left
    elif direction == "right":
        shifted[:, 1:] = image_2d[:, :-1]  # Shift columns right

    # Flatten the shifted image back to 1D
    return shifted.flatten()


def augment_dataset(X, y):
    """
    Augment the MNIST dataset by creating shifted versions of each image.

    Args:
        X (pd.DataFrame): Dataset with features (flattened MNIST images).
        y (pd.Series): Labels for the images.

    Returns:
        pd.DataFrame, pd.Series: Augmented dataset (features and labels).
    """
    augmented_X = []
    augmented_y = []

    directions = ["up", "down", "left", "right"]
    total_iterations = len(X)

    # Iterate through each image and label
    with tqdm.tqdm(total=total_iterations, desc="Augmenting Data") as pbar:
        for image, label in zip(X.values, y.values):
            # Append the original image
            augmented_X.append(image)
            augmented_y.append(label)

            # Create shifted images for all four directions
            for direction in directions:
                shifted_image = shift_image(image, direction)
                augmented_X.append(shifted_image)
                augmented_y.append(label)  # Same label for shifted image
            pbar.update(1)

    print("converting lists to DataFrame and Series")
    # Convert lists to DataFrame and Series
    augmented_X = pd.DataFrame(augmented_X, columns=X.columns)
    augmented_y = pd.Series(augmented_y)

    return augmented_X, augmented_y

## Read Data

In [5]:
raw_data = pd.read_csv(data_root + '/raw_mnist_data.csv')

In [6]:
raw_data.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [7]:
raw_data.shape

(70000, 785)

## Train, Validation & Test Split

In [8]:
## split the data into variables and target
X = raw_data.drop(columns=['class'])
y = raw_data['class']


In [9]:
## split train, validate and test data splits
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, X_test.shape

((44800, 784), (11200, 784), (14000, 784))

## Augment Data

In [12]:
augmented_X, augmented_y = augment_dataset(X_train, y_train)

Augmenting Data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44800/44800 [00:01<00:00, 25133.70it/s]


converting lists to DataFrame and Series


In [13]:
augmented_X.shape, augmented_y.shape

((224000, 784), (224000,))

In [14]:
## save the augmented data
augmented_data = pd.concat([augmented_X, augmented_y], axis=1)
augmented_data.to_csv(data_root + '/augmented_ensemle_learning_mnist_data.csv', index=False)

In [10]:
## read the augmented data
augmented_data = pd.read_csv(data_root + '/augmented_ensemle_learning_mnist_data.csv')

## split the data into variables and target
augmented_X = augmented_data.drop(columns=['0'])
augmented_y = augmented_data['0']

## Train Models using default params

In [11]:
model_dir = project_root + '/models/ensemble/'


### Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

# initialize LogisticRegression
logistic_regression = LogisticRegression(n_jobs=-1, random_state=42, max_iter=10000)

# create pipeline
logistic_regression_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("logistic_regression", logistic_regression)
])

In [17]:
## fit the model
start_time = time.time()
logistic_regression_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 288.37s


In [18]:
## save the model
saved_location = joblib.dump(logistic_regression_pipeline, model_dir + 'logistic_regression_model.pkl')

### SVC

In [19]:
svc_base = SVC(probability=True, random_state=42)  # Enable probability for AUC

# create pipeline
default_svc_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("svc", svc_base)
])

In [20]:
start_time = time.time()
default_svc_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 12811.00s


In [21]:
## save the model
saved_location = joblib.dump(default_svc_pipeline, model_dir + 'svc_model.pkl')

### Random Forest

In [22]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)

# Create a pipeline
rfc_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("randomforest", rfc)
])

In [23]:
start_time = time.time()
rfc_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 21.82s


In [24]:
## save the model
saved_location = joblib.dump(rfc_pipeline, model_dir + 'random_forest_model.pkl')

### KNN

In [25]:
knn = KNeighborsClassifier( n_jobs=-1)

# Create a pipeline
knn_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", knn)
])

In [26]:
start_time = time.time()
knn_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 2.38s


In [27]:
## save the model
saved_location = joblib.dump(knn_pipeline, model_dir + 'knn_model.pkl')

### Extra Trees Classifier

In [28]:
extra_trees = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# Create a pipeline
extra_trees_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("extra_trees", extra_trees)
])

start_time = time.time()
extra_trees_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 28.31s


In [29]:
## save the model
saved_location = joblib.dump(extra_trees_pipeline, model_dir + 'extra_trees_model.pkl')

### Gradient Boosting Classifier

In [30]:
## Gradient Boosting Classifier
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Create a pipeline
gb_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("gb", gb)
])

start_time = time.time()
gb_pipeline.fit(augmented_X, augmented_y)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 6551.28s


In [31]:
## save the model
saved_location = joblib.dump(gb_pipeline, model_dir + 'gradient_boosting_model.pkl')

### Ensemble with Voting Classifier (Hard Voting)


In [ ]:
## read all saved details in case the notebook is restarted
model_dir = project_root + '/models/ensemble/'
data_dir = project_root + '/data/'

## read the augmented data from augmented_ensemle_learning_mnist_data.csv
augmented_data = pd.read_csv(data_dir + 'augmented_ensemle_learning_mnist_data.csv')

## split the data into variables and target
augmented_X = augmented_data.drop(columns=['0'])
augmented_y = augmented_data['0']


## read estimators from saved models
logistic_regression = joblib.load(model_dir + 'logistic_regression_model.pkl')
svc = joblib.load(model_dir + 'svc_model.pkl')
random_forest = joblib.load(model_dir + 'random_forest_model.pkl')
knn = joblib.load(model_dir + 'knn_model.pkl')
extra_trees = joblib.load(model_dir + 'extra_trees_model.pkl')
gradient_boosting = joblib.load(model_dir + 'gradient_boosting_model.pkl')  

In [33]:
voting_clf = VotingClassifier(
    estimators=[
        ("LogReg", logistic_regression),
        ("SVC", svc),
        ("RF", random_forest),
        ("KNN", knn),
        ("ExtraTrees", extra_trees),
        ("GB", gradient_boosting)
    ],
    voting="hard"  # Use "soft" if all models support probability predictions
)

* fit (Optional but Necessary for scikit-learn VotingClassifier): While the models are pre-trained, the VotingClassifier still needs to be "fitted" to understand the labels (classes). 
* We can fit it with a small dummy dataset, or an empty dataset, or even just pass an empty array for X and y


In [34]:
start_time = time.time()
voting_clf.fit(X_train, y_train)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 2108.68s


In [35]:
## save the model
saved_location = joblib.dump(voting_clf, model_dir + 'voting_classifier_model.pkl')

### Prediction

In [36]:
## predict on the validation set
y_val_pred = voting_clf.predict(X_val)

In [37]:
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}

In [38]:
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)


Accuracy: 0.9721428571428572
Weighted F1: 0.9720921798301937
Per-class F1 scores:  {'Class_0': 0.9764192139737992, 'Class_1': 0.9886851346078814, 'Class_2': 0.9684400360685302, 'Class_3': 0.9691438504997827, 'Class_4': 0.9760506100316313, 'Class_5': 0.9781659388646288, 'Class_6': 0.9752747252747253, 'Class_7': 0.9704090513489991, 'Class_8': 0.9644341801385681, 'Class_9': 0.9511153298528714}


### Ensemble with Voting Classifier (Soft Voting)


Observations:
* So the `Accuracy` and `F1 Scores`

In [39]:
soft_voting_clf = VotingClassifier(
    estimators=[
        ("LogReg", logistic_regression),
        ("SVC", svc),
        ("RF", random_forest),
        ("KNN", knn),
        ("ExtraTrees", extra_trees),
        ("GB", gradient_boosting)
    ],
    voting="soft"  # Use "soft" if all models support probability predictions
)

In [40]:
start_time = time.time()
soft_voting_clf.fit(X_train, y_train)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")

Training time: 2094.59s


In [41]:
## save the model
saved_location = joblib.dump(soft_voting_clf, model_dir + 'soft_voting_classifier_model.pkl')

In [42]:
## predict on the validation set
y_val_pred = soft_voting_clf.predict(X_val)

In [43]:
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}

In [44]:
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)

Accuracy: 0.9729464285714285
Weighted F1: 0.9729077740577333
Per-class F1 scores:  {'Class_0': 0.9811155028546333, 'Class_1': 0.9852140077821012, 'Class_2': 0.9712984054669704, 'Class_3': 0.9698558322411533, 'Class_4': 0.9759637188208616, 'Class_5': 0.9744084983099952, 'Class_6': 0.9744292237442922, 'Class_7': 0.9722463139635733, 'Class_8': 0.9677716390423573, 'Class_9': 0.9543529411764706}


Observations:
* So voting classifier significantly outperforms most of the individual default estimators.
* The closest model was default `SVC`, but its still not better then tuned `SVM` that we ended up using in prod, which had accuract and F1 score of .9897. 
* Next step would be to tune all the models and create an ensemble using the best model.
    * For that we'll tune all the models using GridSearch except SVC, since we already have best params for that and training is time consuming. 

## Hyper-param Tuning

* In order to save time, on training models and focusing on learning, we are going to train the models using the best params that we found in previous work sheet. 
* Even though the best params were found on non-augmented data, they are likely to perform well on augmented data too.
* We can add hyper-param tuning as future enhancements

### Logistic Regression

In [45]:
## Below are the hyperparameters for the models
#  Preprocessing: `normalize`
#  Solver : `newton-cg`
#  C : `0.1`
#  Penalty : `l2`
# initialize LogisticRegression
logistic_regression = LogisticRegression(solver="newton-cg", C=0.1, penalty="l2", n_jobs=-1, random_state=42, max_iter=10000)

# create pipeline
tuned_logistic_regression_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("logisticregression", logistic_regression)
])

In [46]:
## start timer

start = time.time()
tuned_logistic_regression_pipeline.fit(augmented_X, augmented_y)
end = time.time()
print(f"Training time: {end - start:.2f}s")

Training time: 131.65s


In [47]:
## save the model
saved_location = joblib.dump(tuned_logistic_regression_pipeline, model_dir + 'tuned_logistic_regression_model.pkl')

In [48]:
## predict on the validation set
y_val_pred = tuned_logistic_regression_pipeline.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)


Accuracy: 0.9197321428571429
Weighted F1: 0.9194301831211195
Per-class F1 scores:  {'Class_0': 0.9606986899563319, 'Class_1': 0.9517660463349791, 'Class_2': 0.9100185528756958, 'Class_3': 0.9082853345148427, 'Class_4': 0.9278996865203761, 'Class_5': 0.8815212091662603, 'Class_6': 0.9409610983981693, 'Class_7': 0.9341527655838455, 'Class_8': 0.872794800371402, 'Class_9': 0.8969024503005085}


### SVM

In [49]:
## training svc on the augmented data using best hyperparameters
svc = SVC(C=10, gamma="scale", kernel="rbf", random_state=42, probability=True)

# create pipeline

print("Creating pipeline...")
svc_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("svc", svc)
], verbose=True)

Creating pipeline...


In [50]:
start_time = time.time()
svc_pipeline.fit(augmented_X, augmented_y.values.ravel())
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f}s")


[Pipeline] ............ (step 1 of 2) Processing scaler, total=   1.1s
[Pipeline] .............. (step 2 of 2) Processing svc, total=250.8min
Training time: 15048.24s


In [51]:
## save the model
saved_location = joblib.dump(svc_pipeline, model_dir + 'tuned_svc_model.pkl')

In [52]:
## predict on the validation set
y_val_pred = svc_pipeline.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)


Accuracy: 0.98875
Weighted F1: 0.9887464218354948
Per-class F1 scores:  {'Class_0': 0.9911347517730497, 'Class_1': 0.9949159170903402, 'Class_2': 0.9882459312839059, 'Class_3': 0.9855579868708971, 'Class_4': 0.9873303167420815, 'Class_5': 0.9889049686444766, 'Class_6': 0.9876543209876543, 'Class_7': 0.9886759581881533, 'Class_8': 0.9876316994961063, 'Class_9': 0.9864549276039234}


### Random Forest

In [53]:
## below are the best hyperparameters for the random forest classifier
## Best Parameters: {'preprocessing__kw_args': {'method': 'normalize'}, 'randomforest__max_depth': None, 'randomforest__min_samples_leaf': 1, 'randomforest__min_samples_split': 2, 'randomforest__n_estimators': 200}

rfc = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200, random_state=42, n_jobs=-1)

tuned_rfc_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("randomforest", rfc)
])

start = time.time()
tuned_rfc_pipeline.fit(augmented_X, augmented_y)
end = time.time()
print(f"Training time: {end - start:.2f}s")

## save the model
saved_location = joblib.dump(tuned_rfc_pipeline, model_dir + 'tuned_random_forest_model.pkl')

## predict on the validation set
y_val_pred = tuned_rfc_pipeline.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)


Training time: 35.12s
Accuracy: 0.9798214285714286
Weighted F1: 0.9798328131009487
Per-class F1 scores:  {'Class_0': 0.9850088183421517, 'Class_1': 0.9901768172888016, 'Class_2': 0.9805517865219358, 'Class_3': 0.9781659388646288, 'Class_4': 0.9808743169398907, 'Class_5': 0.9845111326234269, 'Class_6': 0.9790145985401459, 'Class_7': 0.981230903535574, 'Class_8': 0.9743824336688015, 'Class_9': 0.9624129930394432}


### KNN

In [20]:
## Best KNN Params are 
## {'knn__algorithm': 'auto', 'knn__n_neighbors': 5, 'knn__p': 2, 'knn__weights': 'distance'}

knn = KNeighborsClassifier(algorithm='auto', n_neighbors=5, p=2, weights='distance', n_jobs=-1)

tuned_knn_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", knn)
])

start = time.time()
tuned_knn_pipeline.fit(augmented_X, augmented_y)
end = time.time()

print(f"Training time: {end - start:.2f}s")
## save the model
saved_location = joblib.dump(tuned_knn_pipeline, model_dir + 'tuned_knn_model.pkl')

## predict on the validation set
y_val_pred = tuned_knn_pipeline.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)

Training time: 3.10s
Accuracy: 0.97875
Weighted F1: 0.9787129524087814
Per-class F1 scores:  {'Class_0': 0.9846153846153847, 'Class_1': 0.9841637697952877, 'Class_2': 0.9803383630544125, 'Class_3': 0.9812472743131269, 'Class_4': 0.9803921568627451, 'Class_5': 0.9806949806949807, 'Class_6': 0.9809264305177112, 'Class_7': 0.9813449023861172, 'Class_8': 0.9661971830985916, 'Class_9': 0.9656453110492108}


### Extra Trees Classifier

* We don't have best params for `Extra Trees Classifier` so we'll need to run grid search for this. 
* We'll try to see if we can fit and find the best params quickly else we'll skip this model

In [14]:
param_grid = {
    "extra_trees__n_estimators": [100, 200, 300],
    "extra_trees__max_depth": [None, 10, 20, 30],
    "extra_trees__min_samples_split": [2, 5, 10],
    "extra_trees__min_samples_leaf": [1, 2, 4],
    "extra_trees__bootstrap": [True, False]
}

extra_trees = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# Create a pipeline
extra_trees_pipeline = Pipeline([
    ("scaler", MinMaxScaler()),
    ("extra_trees", extra_trees)
])

## important thing to notice here, setting n_jobs=-1 in RandomizedSearchCV will cause the notebook to crash.
extra_trees_grid_search = GridSearchCV(extra_trees_pipeline, param_grid, cv=3, verbose=1 , n_jobs=4)


start = time.time()
extra_trees_grid_search.fit(augmented_X, augmented_y)
end = time.time()
print(f"Training time: {end - start:.2f}s")



Fitting 3 folds for each of 216 candidates, totalling 648 fits
Training time: 13793.25s


* Just a interesting note, all the time invested in setting up jupyter server on a gaming machine and moving the training there, paid off. The above grid search took ~4 hours of training, but previously it didn't finish training for 12+ hours

In [15]:
## print the best parameters
print("Best Parameters: ", extra_trees_grid_search.best_params_)
## print the best score
print("Best Score: ", extra_trees_grid_search.best_score_)

Best Parameters:  {'extra_trees__bootstrap': False, 'extra_trees__max_depth': None, 'extra_trees__min_samples_leaf': 1, 'extra_trees__min_samples_split': 2, 'extra_trees__n_estimators': 300}
Best Score:  0.9723169649422987


In [16]:
## save the model
saved_location = joblib.dump(extra_trees_grid_search.best_estimator_, model_dir + 'tuned_extra_trees_model.pkl')

In [23]:
Path(model_dir + 'tuned_extra_trees_model.pkl').relative_to(model_root)

PosixPath('ensemble/tuned_extra_trees_model.pkl')

In [25]:
## upload the model to hugging face
path_or_fileobj = Path(model_dir + 'tuned_extra_trees_model.pkl')
path_in_repo = path_or_fileobj.relative_to(model_root)
api.upload_file(repo_id=repo_id, path_or_fileobj=path_or_fileobj,path_in_repo=str(path_in_repo), token=HF_TOKEN, repo_type="model")

tuned_extra_trees_model.pkl:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gaurangdave/mnist_digits_recognition/commit/7c8a68208b45637713d7151c7a03f704b02137b7', commit_message='Upload ensemble/tuned_extra_trees_model.pkl with huggingface_hub', commit_description='', oid='7c8a68208b45637713d7151c7a03f704b02137b7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gaurangdave/mnist_digits_recognition', endpoint='https://huggingface.co', repo_type='model', repo_id='gaurangdave/mnist_digits_recognition'), pr_revision=None, pr_num=None)

In [26]:
## predict on the validation set
y_val_pred = extra_trees_grid_search.best_estimator_.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)

Accuracy: 0.9810714285714286
Weighted F1: 0.9810816413993403
Per-class F1 scores:  {'Class_0': 0.9876434245366285, 'Class_1': 0.9913657770800628, 'Class_2': 0.9791099000908265, 'Class_3': 0.9773123909249564, 'Class_4': 0.9836214740673339, 'Class_5': 0.9845261121856866, 'Class_6': 0.98359161349134, 'Class_7': 0.9825174825174825, 'Class_8': 0.9739368998628258, 'Class_9': 0.9652294853963839}


### Gradient Boosting Classifier

In [12]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 1],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 0.9, 1.0],
    "max_features": [None, "sqrt", "log2"]
}

gb_clf = GradientBoostingClassifier(random_state=42)

# Create a pipeline
# gb_pipeline = Pipeline([
#     ("scaler", MinMaxScaler()),
#     ("gb", gb_clf)
# ])

gb_grid_search = RandomizedSearchCV(gb_clf, param_grid, cv=3, verbose=1,  n_jobs=10, n_iter=20,random_state=42)

start = time.time()
gb_grid_search.fit(augmented_X, augmented_y)
end = time.time()
print(f"Training time: {end - start:.2f}s")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Training time: 25558.94s


In [ ]:
## print the best parameters
print("Best Parameters: ", gb_grid_search.best_params_)
## print the best score
print("Best Score: ", gb_grid_search.best_score_)


Best Parameters:  {'subsample': 1.0, 'n_estimators': 300, 'max_features': 'sqrt', 'max_depth': 7, 'learning_rate': 0.1}
Best Score:  0.9789017826243046


In [14]:
## save the model
saved_location = joblib.dump(gb_grid_search.best_estimator_, model_dir + 'tuned_gradient_boosting_model.pkl')

In [15]:
## upload the model to hugging face
path_or_fileobj = Path(model_dir + 'tuned_gradient_boosting_model.pkl')
path_in_repo = path_or_fileobj.relative_to(model_root)
api.upload_file(repo_id=repo_id, path_or_fileobj=path_or_fileobj,path_in_repo=str(path_in_repo), token=HF_TOKEN, repo_type="model")

tuned_gradient_boosting_model.pkl:   0%|          | 0.00/66.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gaurangdave/mnist_digits_recognition/commit/b7cdd390f3d2ae7719a6e803250d3758b67927c9', commit_message='Upload ensemble/tuned_gradient_boosting_model.pkl with huggingface_hub', commit_description='', oid='b7cdd390f3d2ae7719a6e803250d3758b67927c9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gaurangdave/mnist_digits_recognition', endpoint='https://huggingface.co', repo_type='model', repo_id='gaurangdave/mnist_digits_recognition'), pr_revision=None, pr_num=None)

In [16]:
## predict on the validation set
y_val_pred = gb_grid_search.best_estimator_.predict(X_val)
# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
weighted_f1 = f1_score(y_val, y_val_pred, average='weighted')
# Compute per-class F1 scores
per_class_f1_scores = f1_score(y_val, y_val_pred, average=None)
per_class_f1_dict = {f"Class_{i}": score for i, score in enumerate(per_class_f1_scores)}
## print the metrics
print(f"Accuracy: {accuracy}")
print(f"Weighted F1: {weighted_f1}")
print("Per-class F1 scores: ", per_class_f1_dict)

Accuracy: 0.9860714285714286
Weighted F1: 0.9860734485464957
Per-class F1 scores:  {'Class_0': 0.9876325088339223, 'Class_1': 0.9956744003145891, 'Class_2': 0.9823449524671797, 'Class_3': 0.9846827133479212, 'Class_4': 0.9855334538878843, 'Class_5': 0.9893822393822393, 'Class_6': 0.9835164835164835, 'Class_7': 0.9851657940663177, 'Class_8': 0.9836065573770492, 'Class_9': 0.9818012132524498}
